In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import sys
import shutil

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import numpy as np
import pandas as pd

import tensorflow as tf
#from tensorflow.keras.utils import plot_model
from yolo_core.dataset2 import Dataset
from yolo_core.yolov4 import Create_Yolo, compute_loss
from yolo_core.utils import Load_Yolo_model, postprocess_boxes, nms, image_preprocess, activate_list_in_dataframe
from configuration import *

from tqdm.auto import tqdm

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9738995461714020928
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3643801600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5203149176176659233
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


C:\Users\rh987\AppData\Roaming\Python\Python39\site-packages\albumentations\imgaug\transforms.py:346: FutureWarning: This IAAAffine is deprecated. Please use Affine instead
  warnings.warn("This IAAAffine is deprecated. Please use Affine instead", FutureWarning)


In [2]:
# Create checkpoints folder 폴더 생성
if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER):
    os.makedirs(TRAIN_CHECKPOINTS_FOLDER)
    
def main():
    global TRAIN_FROM_CHECKPOINT
    
    # 메모리 과부하 방지 체크
    gpus = tf.config.experimental.list_physical_devices('GPU')
    print(f'GPUs {gpus}')
    if len(gpus) > 0:
        try: tf.config.experimental.set_memory_growth(gpus[0], True)
        except RuntimeError: pass

    # logging 폴더 생성
    if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
    writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

    # 데이터 셋
    trainset = Dataset('train')
    testset = Dataset('test')

    # 스텝 체크
    steps_per_epoch = len(trainset)
    global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
    warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
    total_steps = TRAIN_EPOCHS * steps_per_epoch


    # 모델 호출
    model = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
    # optimizer
    optimizer = tf.keras.optimizers.Adam()

    # train step 정의
    def train_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=True)
            # loss init
            giou_loss=conf_loss=prob_loss=0

            # optimizing process
            grid = 3 if not TRAIN_YOLO_TINY else 2
            for i in range(grid):
                conv, pred = pred_result[i*2], pred_result[i*2+1]
                loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            gradients = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # update learning rate
            global_steps.assign_add(1)
            if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
                lr = global_steps / warmup_steps * TRAIN_LR_INIT
            else:
                lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                    (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
            optimizer.lr.assign(lr.numpy())

            # writing summary data
            with writer.as_default():
                tf.summary.scalar("lr", optimizer.lr, step=global_steps)
                tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
                tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
                tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
                tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
            writer.flush()
            
        return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

    # valid step 정의
    def val_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=False)
            giou_loss=conf_loss=prob_loss=0

            # optimizing process
            grid = 3 if not TRAIN_YOLO_TINY else 2
            for i in range(grid):
                conv, pred = pred_result[i*2], pred_result[i*2+1]
                loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss
            
        return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()
    
    # 최종 점수 확인
    mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)

    # epochs loop
    best_val_loss = 1000 # Best loss init (best weight 일 때 저장)
    for epoch in range(TRAIN_EPOCHS):
        for image_data, target in trainset:
            results = train_step(image_data, target)
            cur_step = results[0]%steps_per_epoch
            
            # print("Step :", cur_step+1)
            if (cur_step+1)%100 == 0:
                print("epoch:{:3d}/{}, step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
                      .format(epoch+1, TRAIN_EPOCHS, cur_step+1, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))
                                
            if (cur_step+1)%4000 == 0 or (cur_step+1) == steps_per_epoch:
                save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, DATA_TYPE)
                model.save_weights(save_directory)

        if len(testset) == 0:
            print("configure TEST options to validate model")
            model.save_weights(os.path.join(TRAIN_CHECKPOINTS_FOLDER, DATA_TYPE))
            continue
        
        count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
        for image_data, target in testset:
            results = val_step(image_data, target)
            count += 1
            giou_val += results[0]
            conf_val += results[1]
            prob_val += results[2]
            total_val += results[3]


        if TRAIN_SAVE_BEST_ONLY and best_val_loss>total_val/count:
            save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, DATA_TYPE+"_best")
            model.save_weights(save_directory)
            best_val_loss = total_val/count     
            
                   
        # writing validate summary data
        with writer.as_default():
            tf.summary.scalar("validate_loss/total_val", total_val/count, step=epoch)
            tf.summary.scalar("validate_loss/giou_val", giou_val/count, step=epoch)
            tf.summary.scalar("validate_loss/conf_val", conf_val/count, step=epoch)
            tf.summary.scalar("validate_loss/prob_val", prob_val/count, step=epoch)
        writer.flush()

            
        print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
              format(giou_val/count, conf_val/count, prob_val/count, total_val/count))
        
main()

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
train length : 8064
test length : 6634
epoch:  1/5, step:  100/4032, lr:0.000002, giou_loss: 130.42, conf_loss:1446.28, prob_loss: 228.05, total_loss:1804.75
something wrong, check preprocess_true_boxes func


In [31]:
!tensorboard --logdir ./logs

^C


In [1]:
"http://localhost:6006/"

'http://localhost:6006/'

# mAP

In [35]:
# # Test dataset의 ground-truth txt 생성

# import cv2

# GT_PATH = "./input/ground-truth/"
# DET_PATH = "./input/detection-results/"

# df = pd.read_csv("./data/test_detection_frame.csv")
# df = activate_list_in_dataframe(df)

# img_path_arr = df["img_path"].values
# x_min_arr = df["x_min"].values
# x_max_arr = df["x_max"].values
# y_min_arr = df["y_min"].values
# y_max_arr = df["y_max"].values
# class_arr = df["classes"].values

# for idx in tqdm(range(len(df))):
#     name = os.path.split(img_path_arr[idx])[1]
#     name = os.path.splitext(name)[0]
#     GT_OUTPUT = os.path.join(GT_PATH, f"{name}.txt")

#     f = open(GT_OUTPUT, 'w')
#     for b_idx in range(len(x_min_arr[idx])):
#         write_data = []
#         write_data.append(str(class_arr[idx][b_idx]))
#         write_data.append(str(x_min_arr[idx][b_idx]))
#         write_data.append(str(y_min_arr[idx][b_idx]))
#         write_data.append(str(x_max_arr[idx][b_idx]))
#         write_data.append(str(y_max_arr[idx][b_idx]))
#         write_data = " ".join(write_data)+"\n"
#         f.write(write_data)
#     f.close()

  0%|          | 0/6635 [00:00<?, ?it/s]

In [3]:
import cv2

DET_PATH = "./input/detection-results/"

df = pd.read_csv("./data/test_detection_frame.csv")
df = activate_list_in_dataframe(df)
ckpt_directory = "checkpoints/yolov4_33_ciou/visdrone_best"
yolo = Load_Yolo_model(ckpt_directory)

img_path_arr = df["img_path"].values

for idx in tqdm(range(len(df))):
    name = os.path.split(img_path_arr[idx])[1]
    name = os.path.splitext(name)[0]
    DET_OUTPUT = os.path.join(DET_PATH, f"{name}.txt")

    original_image      = cv2.imread(img_path_arr[idx])
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    image_data = image_preprocess(np.copy(original_image), [416, 416])
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    pred_bbox = yolo.predict_step(image_data)
        
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    bboxes = postprocess_boxes(pred_bbox, original_image, 416, 0.3)
    bboxes = nms(bboxes, 0.45, method='nms')

    f = open(DET_OUTPUT, 'w')
    for pred in bboxes:
        pred = np.roll(pred, 2)
        pred[0], pred[1] = pred[1], pred[0]
        pred = pred.tolist()
        pred[0] = int(pred[0])
        pred[2] = int(pred[2])
        pred[3] = int(pred[3])
        pred[4] = int(pred[4])
        pred[5] = int(pred[5])
        write_data = " ".join([str(i) for i in pred])+"\n"
        # print(write_data)
        f.write(write_data)
    f.close()

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


ValueError: Received incompatible tensor with shape (1, 1, 64, 64) when attempting to restore variable with shape (1, 1, 128, 64) and name layer_with_weights-10/kernel/.ATTRIBUTES/VARIABLE_VALUE.

In [30]:
!python mAP.py -na

17.40% = 0 AP 
16.08% = 1 AP 
52.09% = 2 AP 
mAP = 28.52%
Figure(640x480)


# Pred Box

In [31]:
from glob import glob

RESULT_PATH = "./result/baseline_yolov3_33" # 정리된 annotations
PRED_PATH = "./input/yolov3_33/" + "*" # mAP를 구할때 뽑았던 pred_annotations

if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)

video_name_list = []
for path in glob(PRED_PATH):
    _, name = os.path.split(path)
    video_name = "_".join(name.split("_")[:3])
    video_name_list.append(video_name)
video_name_list = list(set(video_name_list))

for video_name in tqdm(video_name_list):
    DST_PATH = os.path.join(RESULT_PATH,video_name+"_pred.txt")
    w_file = open(DST_PATH, "w")
    video_frame_list = [path for path in glob(PRED_PATH) if os.path.split(path)[-1].startswith(video_name)]
    for idx, video_frame in enumerate(video_frame_list):
        file = open(video_frame,"r")
        strings = file.readlines()
        strings = [line.strip().split() for line in strings]
        for string in strings:
            writer = []
            writer.append(str(idx+1))
            writer.append(string[2])
            writer.append(string[3])
            writer.append(string[4])
            writer.append(string[5])
            writer.append(string[1])
            writer.append(string[0])
            writer = ",".join(writer)+"\n"
            w_file.write(writer)
        file.close()

  0%|          | 0/17 [00:00<?, ?it/s]